# 线性回归简洁实现

即调包

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

还是和之前一样，我们作为上帝先进行真实值定义

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # 调包，直接人工生成数据

调包读取数据：

这里，

```python
data_arrays = [tensor1, tensor2]
dataset = data.TensorDataset(*data_arrays)
```

等价于：

```python
dataset = data.TensorDataset(tensor1, tensor2)
```

这是python中的解包操作

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  
    dataset = data.TensorDataset(*data_arrays)  # 指定数据集
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 批加载、打乱数据集

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))  # 看看第一批数据

[tensor([[ 2.7665,  0.2529],
         [ 1.9997, -1.3722],
         [ 0.5810, -0.8961],
         [ 0.9370, -0.3671],
         [-0.5888,  1.6066],
         [-1.4575,  0.5634],
         [-0.5592,  0.4610],
         [-0.9245, -0.2676],
         [-0.1836, -1.9164],
         [-0.5899, -0.6669]]),
 tensor([[ 8.8739],
         [12.8626],
         [ 8.4085],
         [ 7.3243],
         [-2.4392],
         [-0.6215],
         [ 1.5175],
         [ 3.2718],
         [10.3377],
         [ 5.2948]])]

模型和loss也是预先定义好的：

In [4]:
from torch import nn # nn是神经网络的缩写，有大量定义好的神经网络层

# 这里，我们使用全连接层（Linear），等价于传统的线性回归
net = nn.Sequential(nn.Linear(2, 1))  # 参数：输入维度、输出维度。这里即接收两个特征（x），映射到一个输出（y）
# nn.Sequential 是一个容器模块，它按顺序将网络层组合在一起，可以把多个层堆叠在一起构建复杂的网络结构
# 但是这里我们只有一层简单的神经网络，所以这里可写可不写

In [5]:
# 初始化参数：
net[0].weight.data.normal_(0, 0.01) # w，设成正太分布的随机值
net[0].bias.data.fill_(0) # b，设成0
# net只有第0个元素

tensor([0.])

In [6]:
# loss
loss = nn.MSELoss()

# sgd
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程还是要手写：

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y) # 模型自带参数，直接传x，和真实y作loss即可
        trainer.zero_grad()  # 先梯度清理，否则会累计
        l.backward()  # pytorch会帮我们做sum() 操作
        trainer.step()  # 参数更新
    l = loss(net(features), labels)
    print(f'epoch {epoch}, loss {l:f}')

epoch 0, loss 0.000308
epoch 1, loss 0.000096
epoch 2, loss 0.000096


In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 1.0097e-04, -9.9421e-05])
b的估计误差： tensor([-0.0001])
